## Тестирование ClickHouse 

### Формируем тестовые данные

In [1]:
import datetime
import csv
import random
import uuid

In [2]:
movie_uuids = [uuid.uuid4() for _ in range(1000)]
user_uuids = [uuid.uuid4() for _ in range(2000)]

In [65]:
with open('csv_movie_10.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    headers = ['id', 'user_uuid', 'movie_uuid', 'movie_progress', 'movie_length', 'event_time']
    writer.writerow(headers)
    for i in range(1, 10000001):
        length = random.randint(600, 90000)
        row = [i, random.choice(user_uuids), random.choice(movie_uuids), random.randint(0, length), length, datetime.datetime.now()]
        writer.writerow(row)
    

### Подключаемся к БД

In [8]:
pip install clickhouse-driver

Note: you may need to restart the kernel to use updated packages.


In [1]:
from clickhouse_driver import Client

client = Client(host='localhost') 

In [20]:
client.execute('SHOW DATABASES')

[('_temporary_and_external_tables',),
 ('analytics',),
 ('default',),
 ('replica',),
 ('shard',),
 ('system',)]

### Запись данных в таблицу (1 000 000 столбцов)

In [10]:
import time
import pandas
from pydantic import parse_obj_as, BaseModel
from typing import List

In [86]:
class MovieView(BaseModel):
    id: int
    user_uuid: str
    movie_uuid: str
    movie_progress: int
    movie_length: int
    event_time: datetime.datetime

In [87]:
df1 = pandas.read_csv('csv_movie_1.csv', header=0)
movies_df1 = df1.to_dict(orient='records')
movies_data = parse_obj_as(List[MovieView], movies_df1)

In [89]:
movies_list = [data.dict() for data in movies_data]

In [102]:
%%time
step = 1000
start = time.time()
for i in range(0, len(df1), step):
    movies = movies_list[i:i+step]
    client.execute('INSERT INTO analytics.movie_view (id, user_uuid, movie_uuid, movie_progress, movie_length, event_time) VALUES', 
                   movies) 
end = time.time()
print('Result:', (end - start)*1000, 'ms.')

Result: 35061.02895736694 ms.
CPU times: total: 3.94 s
Wall time: 35.1 s


### Запросы к таблице (1 000 000 столбцов)

In [103]:
%%time
client.execute('SELECT COUNT(*) FROM analytics.movie_view') 

CPU times: total: 0 ns
Wall time: 69.7 ms


[(1000000,)]

In [105]:
%%time
client.execute('SELECT uniqExact(movie_uuid) FROM analytics.movie_view')

CPU times: total: 0 ns
Wall time: 29.9 ms


[(1000,)]

In [106]:
%%time
client.execute('SELECT uniqExact(user_uuid) FROM analytics.movie_view')

CPU times: total: 0 ns
Wall time: 24.4 ms


[(2000,)]

In [107]:
%%time
client.execute('SELECT uniqExact(movie_uuid) FROM analytics.movie_view WHERE movie_length = movie_progress')

CPU times: total: 0 ns
Wall time: 15.3 ms


[(59,)]

### Запись данных в таблицу (10 000 000 столбцов)

In [109]:
df10 = pandas.read_csv('csv_movie_10.csv', header=0)
movies_df10 = df10.to_dict(orient='records')

In [110]:
movies_data = parse_obj_as(List[MovieView], movies_df10)

In [111]:
movies_list = [data.dict() for data in movies_data]

In [112]:
%%time
step = 1000
start = time.time()
for i in range(0, len(df1), step):
    movies = movies_list[i:i+step]
    client.execute('INSERT INTO analytics.movie_view (id, user_uuid, movie_uuid, movie_progress, movie_length, event_time) VALUES', 
                   movies) 
end = time.time()
print('Result:', (end - start)*1000, 'ms.')

Error on localhost:9000 ping: timed out
Connection was closed, reconnecting.


Result: 743004.3733119965 ms.
CPU times: total: 1min 11s
Wall time: 12min 23s


### Запросы к таблице (11 000 000 столбцов)

In [3]:
%%time
client.execute('SELECT COUNT(*) FROM analytics.movie_view') 

CPU times: total: 15.6 ms
Wall time: 9.26 ms


[(11000000,)]

In [4]:
%%time
client.execute('SELECT uniqExact(movie_uuid) FROM analytics.movie_view')

CPU times: total: 0 ns
Wall time: 138 ms


[(1000,)]

In [5]:
%%time
client.execute('SELECT uniqExact(user_uuid) FROM analytics.movie_view')

CPU times: total: 0 ns
Wall time: 192 ms


[(2000,)]

In [6]:
%%time
client.execute('SELECT uniqExact(movie_uuid) FROM analytics.movie_view WHERE movie_length = movie_progress')

CPU times: total: 0 ns
Wall time: 59.1 ms


[(471,)]